## Analysis of the Quasiparticle Method from an analytical perspective

#### Analysis of the effective one-body matrix

Imports

In [121]:

from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverterOnlynnpp,HardcoreBosonsBasis
from scipy.sparse import lil_matrix
file_name='data/kb3g.a42'

SPS=SingleParticleState(file_name=file_name)
print(SPS.energies)
nparticles_a=2
nparticles_b=0

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{42}$Ca'

[0.  0.  0.  0.  0.  0.  0.  0.  2.  2.  2.  2.  6.5 6.5 6.5 6.5 6.5 6.5
 4.  4.  0.  0.  0.  0.  0.  0.  0.  0.  2.  2.  2.  2.  6.5 6.5 6.5 6.5
 6.5 6.5 4.  4. ]


NSM Hamiltonian

In [122]:

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b]-300/2)
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/kb3g_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)
for b in NSMHamiltonian.basis:
    print(b,'\n')

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 40 20
[-302.69198484]
[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 

Quasiparticle Basis

In [123]:
QPC=QuasiParticlesConverterOnlynnpp()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

print(QPC.particles2quasiparticles.shape)

for b in QPC.quasiparticle_basis:
    print(b,'\n')

(10, 30)
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 



Get the effective Hamiltonian

In [124]:

hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_qr=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_rq=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

values,psi=eigsh(hamiltonian_rr,k=1)
e0=values[0]

print(values[0])


values,psi=eigsh(hamiltonian_qq,k=1)
e0=values[0]

print(values[0])
values,psi=eigsh(hamiltonian_qq+(1/e0)*hamiltonian_qr @ hamiltonian_rq,k=1)

e1=values[0]

values,psi=eigsh(hamiltonian_qq+(1/e1)*hamiltonian_qr @ hamiltonian_rq+(1/e1**2)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rq,k=1)

e2=values[0]
values,psi=eigsh(hamiltonian_qq+(1/e2)*hamiltonian_qr @ hamiltonian_rq+(1/e2**2)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rq+(1/e2**3)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rr @ hamiltonian_rq,k=1)


print(values[0],np.abs((values[0]-egs[0])/egs[0]))

print(hamiltonian_qq)

-298.7681339767397
-302.6919848373527
-302.6919848373526 7.511717747181446e-16
  (0, 0)	-301.14469696969695
  (0, 1)	0.5215151515151517
  (0, 2)	-0.18227272727272736
  (0, 3)	0.07151515151515167
  (0, 4)	-0.4221089088238428
  (0, 5)	0.1315557008452238
  (0, 6)	-1.2234115238523078
  (0, 7)	0.6017168243708023
  (0, 8)	-0.5893504775279041
  (0, 9)	-0.357
  (1, 0)	0.5215151515151517
  (1, 1)	-300.56313852813855
  (1, 2)	0.5264935064935063
  (1, 3)	-0.3088528138528137
  (1, 4)	0.2975861054044347
  (1, 5)	-0.2560785042646319
  (1, 6)	0.7793438813655185
  (1, 7)	-1.0316514990371368
  (1, 8)	0.6034834453483592
  (1, 9)	0.357
  (2, 0)	-0.18227272727272736
  (2, 1)	0.5264935064935063
  (2, 2)	-300.4918831168831
  (2, 3)	0.7193506493506493
  (2, 4)	-0.2145709031248292
  :	:
  (7, 5)	-0.33117486666552504
  (7, 6)	0.3245714285714284
  (7, 7)	-287.1002857142857
  (7, 8)	0.43514285714285705
  (7, 9)	0.2263213055223333
  (8, 0)	-0.5893504775279041
  (8, 1)	0.6034834453483592
  (8, 2)	-0.89717304570280

Get the effective coupling

Build $H_{qq}$ using qubits

In [ ]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.
for i in range(len(QPC.couples)//2): #just n-n 
    for j in range(len(QPC.couples)//2):
        
        
        
        if i==j:
            h_qq+=(hamiltonian_qq[i,j]-external_potential_qq[i,j])*QubitBasis.adag_a_matrix(i,j)
        else:
            h_qq+=hamiltonian_qq[i,j]*QubitBasis.adag_a_matrix(i,j)



one_body={}
for i,b in enumerate(QPC.quasiparticle_basis):
    for j,d in enumerate(QPC.quasiparticle_basis):
        idx_b=np.nonzero(b)[0]
        idx_d=np.nonzero(d)[0]
        print(idx_d,idx_b)
        one_body[(idx_b[0],idx_d[0])]=hamiltonian_qq[i,j]
        one_body[(idx_b[0]+len(QPC.couples)//2,idx_d[0]+len(QPC.couples)//2)]=hamiltonian_qq[i,j]

print(one_body.keys())

NotImplementedError: subtracting a nonzero scalar from a sparse matrix is not supported

#### Two-body term (nn)--(nn)

In [112]:
file_name='data/kb3g.a42'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=4
nparticles_b=0

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{20}$O'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b]-300/4)
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/kb3g_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 40 20
[-305.00587205]
total_m= [-2.77555756e-17]


Pairing Hamiltonian one-body term

In [113]:
QPC=QuasiParticlesConverterOnlynnpp()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

print(QPC.particles2quasiparticles.shape)
print(QPC.quasiparticle_basis)

print(QPC.rest_basis.shape)
print(QPC.couples)
print(QPC.rest_basis)

external_potential_qubit=np.zeros(len(QPC.couples))

external_potential_qubit=2*SPS.energies


(45, 565)
[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0

#### Two-body correction

In [ ]:

hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_qr=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_rq=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
#print(twobody_qr)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

#### twobody terms
twobody_nn={}
for i,b in enumerate(QPC.quasiparticle_basis):
    for j,d in enumerate(QPC.quasiparticle_basis):
        idx_a,idx_b=np.nonzero(b)[0]
        idx_c,idx_d=np.nonzero(d)[0]
        if hamiltonian_qq[i,j]-h_qq[i,j]!=0.:
            twobody_nn[(idx_a,idx_b,idx_c,idx_d)]=hamiltonian_qq[i,j]-h_qq[i,j]
              

 

h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

value,_=eigsh(h_qq,k=1)
print(h_qq)

print(value)
print(twobody_nn)

dict_keys([(0, 0), (10, 10), (0, 1), (10, 11), (0, 2), (10, 12), (0, 3), (10, 13), (0, 4), (10, 14), (0, 5), (10, 15), (0, 6), (10, 16), (0, 7), (10, 17), (0, 8), (10, 18), (0, 9), (10, 19), (1, 0), (11, 10), (1, 1), (11, 11), (1, 2), (11, 12), (1, 3), (11, 13), (1, 4), (11, 14), (1, 5), (11, 15), (1, 6), (11, 16), (1, 7), (11, 17), (1, 8), (11, 18), (1, 9), (11, 19), (2, 0), (12, 10), (2, 1), (12, 11), (2, 2), (12, 12), (2, 3), (12, 13), (2, 4), (12, 14), (2, 5), (12, 15), (2, 6), (12, 16), (2, 7), (12, 17), (2, 8), (12, 18), (2, 9), (12, 19), (3, 0), (13, 10), (3, 1), (13, 11), (3, 2), (13, 12), (3, 3), (13, 13), (3, 4), (13, 14), (3, 5), (13, 15), (3, 6), (13, 16), (3, 7), (13, 17), (3, 8), (13, 18), (3, 9), (13, 19), (4, 0), (14, 10), (4, 1), (14, 11), (4, 2), (14, 12), (4, 3), (14, 13), (4, 4), (14, 14), (4, 5), (14, 15), (4, 6), (14, 16), (4, 7), (14, 17), (4, 8), (14, 18), (4, 9), (14, 19), (5, 0), (15, 10), (5, 1), (15, 11), (5, 2), (15, 12), (5, 3), (15, 13), (5, 4), (15, 14),

#### The $^{24}$ Mg case

In [115]:
file_name='data/kb3g.a42'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=6
nparticles_b=0

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{28}$Si'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b]-300/6)
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/kb3g_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis)) 
        
        
        



size= 40 20
[-306.78800292]
total_m= [1.08010253e-16]


Build the corresponding $H_{qq}$ using qubits

In [116]:
QPC=QuasiParticlesConverterOnlynnpp()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

for r in QPC.rest_basis:
    print(r,'\n')


[1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0] 

[1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 

check the $H_{QQ}$

In [119]:
hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
value,_=eigsh(hamiltonian_qq,k=1)

print(value[0])
print((value[0]-egs)/egs)

print('norm ham_rr=',scipy.sparse.linalg.norm(hamiltonian_rr))
value,_=eigsh(hamiltonian_rr,k=1)
print(value[0])
print(hamiltonian_rr.shape)

print(hamiltonian_qq)

-306.50849295230444
[-0.00091109]
norm ham_rr= 17546.315547215014
-302.7103782539687
(3832, 3832)
  (0, 0)	-304.19028138528137
  (0, 1)	0.7193506493506493
  (0, 2)	-0.2145709031248292
  (0, 3)	0.33909370654423737
  (0, 4)	-0.6254325972780652
  (0, 5)	0.8918731827701394
  (0, 6)	-0.8971730457028098
  (0, 7)	-0.357
  (0, 8)	-0.3088528138528137
  (0, 9)	0.2975861054044347
  (0, 10)	-0.2560785042646319
  (0, 11)	0.7793438813655185
  (0, 12)	-1.0316514990371368
  (0, 13)	0.6034834453483592
  (0, 14)	0.357
  (0, 36)	0.07151515151515167
  (0, 37)	-0.4221089088238428
  (0, 38)	0.1315557008452238
  (0, 39)	-1.2234115238523078
  (0, 40)	0.6017168243708023
  (0, 41)	-0.5893504775279041
  (0, 42)	-0.357
  (1, 0)	0.7193506493506493
  (1, 1)	-303.8618831168831
  (1, 2)	0.17306330198502648
  :	:
  (118, 117)	0.43514285714285705
  (118, 118)	-265.19861904761905
  (118, 119)	0.3245714285714284
  (119, 33)	-0.357
  (119, 34)	-0.5893504775279041
  (119, 35)	0.6017168243708023
  (119, 61)	0.357
  (119, 62

In [56]:
print(np.sqrt(8015/396))

4.498877525550011


(if you want) adding the tree level correction (diagonal)

In [120]:
#### twobody terms
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.
print(one_body)
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    

print(twobody_nn)


value,_=eigsh(h_qq,k=1)

print(value)
print(h_qq)



{(0, 0): -301.14469696969695, (10, 10): -301.14469696969695, (0, 1): 0.5215151515151517, (10, 11): 0.5215151515151517, (0, 2): -0.18227272727272736, (10, 12): -0.18227272727272736, (0, 3): 0.07151515151515167, (10, 13): 0.07151515151515167, (0, 4): -0.4221089088238428, (10, 14): -0.4221089088238428, (0, 5): 0.1315557008452238, (10, 15): 0.1315557008452238, (0, 6): -1.2234115238523078, (10, 16): -1.2234115238523078, (0, 7): 0.6017168243708023, (10, 17): 0.6017168243708023, (0, 8): -0.5893504775279041, (10, 18): -0.5893504775279041, (0, 9): -0.357, (10, 19): -0.357, (1, 0): 0.5215151515151517, (11, 10): 0.5215151515151517, (1, 1): -300.56313852813855, (11, 11): -300.56313852813855, (1, 2): 0.5264935064935063, (11, 12): 0.5264935064935063, (1, 3): -0.3088528138528137, (11, 13): -0.3088528138528137, (1, 4): 0.2975861054044347, (11, 14): 0.2975861054044347, (1, 5): -0.2560785042646319, (11, 15): -0.2560785042646319, (1, 6): 0.7793438813655185, (11, 16): 0.7793438813655185, (1, 7): -1.031651

In [96]:
for key,value in twobody_nn.items():
    a,b,c,d=key
    if a<6 and b<6 and c<6 and d<6:
        print(a,b,c,d,'\n')

print(twobody_nn.keys())

0 1 0 1 

0 2 0 2 

0 3 0 3 

0 4 0 4 

0 5 0 5 

1 2 1 2 

1 3 1 3 

1 4 1 4 

1 5 1 5 

2 3 2 3 

2 4 2 4 

2 5 2 5 

3 4 3 4 

3 5 3 5 

4 5 4 5 

dict_keys([(0, 1, 0, 1), (0, 2, 0, 2), (0, 3, 0, 3), (0, 4, 0, 4), (0, 5, 0, 5), (0, 6, 0, 6), (0, 7, 0, 7), (0, 8, 0, 8), (0, 9, 0, 9), (1, 2, 1, 2), (1, 3, 1, 3), (1, 4, 1, 4), (1, 5, 1, 5), (1, 6, 1, 6), (1, 7, 1, 7), (1, 8, 1, 8), (1, 9, 1, 9), (2, 3, 2, 3), (2, 4, 2, 4), (2, 5, 2, 5), (2, 6, 2, 6), (2, 7, 2, 7), (2, 8, 2, 8), (2, 9, 2, 9), (3, 4, 3, 4), (3, 5, 3, 5), (3, 6, 3, 6), (3, 7, 3, 7), (3, 8, 3, 8), (3, 9, 3, 9), (4, 5, 4, 5), (4, 6, 4, 6), (4, 7, 4, 7), (4, 8, 4, 8), (4, 9, 4, 9), (5, 6, 5, 6), (5, 7, 5, 7), (5, 8, 5, 8), (5, 9, 5, 9), (6, 7, 6, 7), (6, 8, 6, 8), (6, 9, 6, 9), (7, 8, 7, 8), (7, 9, 7, 9), (8, 9, 8, 9)])


First order

In [97]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.

for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)


# for key,value in four_body.items():
#     idx_a,idx_b,idx_c,idx_d=key
#     h_qq+=value*QubitBasis.four_body_matrix(idx_a,idx_b,idx_c,idx_d,idx_a,idx_b,idx_c,idx_d)
value,_=eigsh(h_qq,k=1)
print(h_qq)
print(value[0])

    


  (0, 0)	-4.1902813852813665
  (0, 1)	0.7193506493506493
  (0, 2)	-0.2145709031248292
  (0, 3)	0.33909370654423737
  (0, 4)	-0.6254325972780652
  (0, 5)	0.8918731827701394
  (0, 6)	-0.8971730457028098
  (0, 7)	-0.357
  (0, 8)	-0.3088528138528137
  (0, 9)	0.2975861054044347
  (0, 10)	-0.2560785042646319
  (0, 11)	0.7793438813655185
  (0, 12)	-1.0316514990371368
  (0, 13)	0.6034834453483592
  (0, 14)	0.357
  (0, 36)	0.07151515151515167
  (0, 37)	-0.4221089088238428
  (0, 38)	0.1315557008452238
  (0, 39)	-1.2234115238523078
  (0, 40)	0.6017168243708023
  (0, 41)	-0.5893504775279041
  (0, 42)	-0.357
  (1, 0)	0.7193506493506493
  (1, 1)	-3.8618831168831775
  (1, 2)	0.17306330198502648
  (1, 3)	-0.3806013076840401
  (1, 4)	0.5911170985054609
  (1, 5)	-0.694063594823274
  (1, 6)	1.1292981324222793
  (1, 7)	0.357
  (1, 8)	0.5264935064935063
  (1, 15)	0.2975861054044347
  (1, 16)	-0.2560785042646319
  (1, 17)	0.7793438813655185
  (1, 18)	-1.0316514990371368
  (1, 19)	0.6034834453483592
  (1, 20

Arbitrary order with a loop

In [22]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
print(external_potential_qq.shape)

h_qq=0.

for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
    
for r,correction in enumerate(two_body_corrections):

    delta_hamiltonian=0.    
    for j in trange(r):
        for keys,values in two_body_corrections[j].items():
            idx_a,idx_b,idx_c,idx_d=keys
            correction_term=values*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)/(e*(e)**(j))
            delta_hamiltonian+=correction_term
    # for j in trange(r):
    #     for keys,values in two_body_corrections_nn[j].items():
    #         idx_a,idx_b,idx_c,idx_d=keys
    #         correction_term=values*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)/(e*(e/1.5)**(j))
    #         delta_hamiltonian+=correction_term
                    
    value,_=eigsh(h_qq+delta_hamiltonian,k=1)
    e=value[0]
    print('iteration=',r,' energy=',e,'\n')



    

(225, 225)


0it [00:00, ?it/s]


iteration= 0  energy= -78.9574772646304 



100%|██████████| 1/1 [00:05<00:00,  5.90s/it]


iteration= 1  energy= -81.29318444796019 



...

iteration= 2  energy= -81.3580916066351 



100%|██████████| 3/3 [00:19<00:00,  6.64s/it]


iteration= 3  energy= -81.38751036560774 



  0%|          | 0/4 [00:04<?, ?it/s]


KeyboardInterrupt: 

#### Other Cases

In [271]:
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=4

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{22}$Ne'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis)) 
        
        
        



size= 24 12
[-57.45408791]
total_m= [4.32321381e-17]


Build the corresponding $H_{qq}$ using qubits

In [272]:
QPC=QuasiParticlesConverter()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)



check the $H_{QQ}$

In [273]:
hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T

value,_=eigsh(hamiltonian_qq,k=1)

print(value[0])

-51.352457221312235


In [274]:
print(hamiltonian_qq)

  (0, 0)	-48.05152222222223
  (0, 1)	0.8598999999999996
  (0, 2)	0.9007818899896537
  (0, 3)	1.133619249748832
  (0, 4)	-1.3995613925794361
  (0, 5)	-0.4062999999999996
  (0, 6)	-0.9007818899896537
  (0, 7)	-1.9314456782406437
  (0, 8)	0.6017349640876244
  (0, 15)	0.6897999999999996
  (0, 30)	-0.4062999999999996
  (0, 45)	-0.900781889989654
  (0, 60)	-1.9314456782406446
  (0, 75)	0.6017349640876247
  (1, 0)	0.8598999999999996
  (1, 1)	-46.40699047619047
  (1, 2)	-0.9007818899896537
  (1, 3)	-0.7347060355029265
  (1, 4)	1.7984746068253417
  (1, 5)	0.6897999999999995
  (1, 9)	-0.9007818899896537
  (1, 10)	-1.9314456782406437
  (1, 11)	0.6017349640876244
  (1, 16)	0.6897999999999996
  (1, 31)	-0.4062999999999996
  :	:
  (88, 58)	0.7177133829043456
  (88, 73)	0.9008999999999998
  (88, 77)	0.6017349640876244
  (88, 79)	-0.9007818899896537
  (88, 81)	-1.3995613925794361
  (88, 83)	0.9007818899896537
  (88, 84)	1.7984746068253417
  (88, 86)	-0.9007818899896537
  (88, 87)	0.9008999999999994
  

(if you want) adding the tree level correction (diagonal)

First order

In [275]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.

for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

# for key,value in four_body.items():
#     idx_a,idx_b,idx_c,idx_d=key
#     h_qq+=value*QubitBasis.four_body_matrix(idx_a,idx_b,idx_c,idx_d,idx_a,idx_b,idx_c,idx_d)
value,_=eigsh(h_qq,k=1)
print(h_qq)
print(value[0])

    


  (0, 0)	-32.348722222222236
  (0, 1)	0.8598999999999996
  (0, 2)	0.9007818899896537
  (0, 3)	1.133619249748832
  (0, 4)	-1.3995613925794361
  (0, 5)	-0.4062999999999996
  (0, 6)	-0.9007818899896537
  (0, 7)	-1.9314456782406437
  (0, 8)	0.6017349640876244
  (0, 15)	0.6897999999999996
  (0, 30)	-0.4062999999999996
  (0, 45)	-0.900781889989654
  (0, 60)	-1.9314456782406446
  (0, 75)	0.6017349640876247
  (1, 0)	0.8598999999999996
  (1, 1)	-30.704190476190497
  (1, 2)	-0.9007818899896537
  (1, 3)	-0.7347060355029265
  (1, 4)	1.7984746068253417
  (1, 5)	0.6897999999999995
  (1, 9)	-0.9007818899896537
  (1, 10)	-1.9314456782406437
  (1, 11)	0.6017349640876244
  (1, 16)	0.6897999999999996
  (1, 31)	-0.4062999999999996
  (1, 46)	-0.900781889989654
  (1, 61)	-1.9314456782406446
  (1, 76)	0.6017349640876247
  (2, 0)	0.9007818899896537
  (2, 1)	-0.9007818899896537
  (2, 2)	-29.156129365079376
  (2, 3)	-0.7177133829043454
  (2, 4)	0.7177133829043454
  (2, 6)	0.6897999999999995
  (2, 9)	-0.40629999

Second order

In [ ]:
h_qq=0.
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        
        h_qq+=one_body[i,j]*QubitBasis.adag_a_matrix(i,j)


for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                h_qq+=(twobody_nnpp[i,j,k,l]+twobody_nn[i,j,k,l]+two_body_corrections[0][i,j,k,l]/value[0])*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                
value,_=eigsh(h_qq+external_potential_qq,k=1)

print(value)

[-53.5305729]


Third order

In [ ]:
h_qq=0.
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        
        h_qq+=one_body[i,j]*QubitBasis.adag_a_matrix(i,j)


for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                h_qq+=(twobody_nnpp[i,j,k,l]+twobody_nn[i,j,k,l]+two_body_corrections[0][i,j,k,l]/value[0]+two_body_corrections[1][i,j,k,l]/value[0]**2)*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                
value,_=eigsh(h_qq+external_potential_qq,k=1)

print(value)

[-54.25552219]


Arbitrary order with a loop

In [ ]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
print(external_potential_qq.shape)

hamiltonian_qq_qubit=0. 

for i in range(len(QPC.couples)): #just neutrons
        for j in range(len(QPC.couples)):
            
            hamiltonian_qq_qubit+=one_body[i,j]*QubitBasis.adag_a_matrix(i,j)
            
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                hamiltonian_qq_qubit+=(twobody_nnpp[i,j,k,l]+twobody_nn[i,j,k,l])*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                #hamiltonian_qq_qubit+=four_body[i,j,k,l]*QubitBasis.four_body_matrix(i,j,k,l,i,j,k,l)

for r,correction in enumerate(two_body_corrections[:5]):

        
    overall_twobody=np.zeros((len(QPC.couples),len(QPC.couples),len(QPC.couples),len(QPC.couples)))
    for j in range(r):
        overall_twobody+=(two_body_corrections[j])/((e)**(j+1))
    delta_hamiltonian=0.
    for i in range(len(QPC.couples)): #just neutrons
        for j in range(len(QPC.couples)):
            for k in range(len(QPC.couples)):
                for l in range(len(QPC.couples)):
                    #print(i,j,k,l)
                    #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                    delta_hamiltonian+=overall_twobody[i,j,k,l]*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                    
    value,_=eigsh(hamiltonian_qq_qubit+delta_hamiltonian,k=1)
    e=value[0]
    print('iteration=',r,' energy=',e,'\n')

delta_fourbody=0.
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                delta_fourbody+=four_body[i,j,k,l]*QubitBasis.four_body_matrix(i,j,k,l,i,j,k,l)
value,_=eigsh(hamiltonian_qq_qubit+delta_hamiltonian+delta_fourbody,k=1)
e=value[0]
print('iteration=',r,' energy=',e,'\n')

    

(225, 225)
iteration= 0  energy= -66.3824753272453 

iteration= 1  energy= -69.75675976421405 

iteration= 2  energy= -70.89029762179018 

iteration= 3  energy= -71.34028995152087 

iteration= 4  energy= -71.53242775338654 

iteration= 4  energy= -79.1134283333448 

